In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Unrecognized magic `%%shell`.

Julia does not use the IPython `%magic` syntax.   To interact with the IJulia kernel, use `IJulia.somefunction(...)`, for example.  Julia macros, string macros, and functions can be used to accomplish most of the other functionalities of IPython magics.


In [ ]:
versioninfo()

Julia Version 1.8.2
Commit 36034abf260 (2022-09-29 15:21 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × AMD EPYC 7B12
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, znver2)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2


Add new code cells by clicking the `+ Code` button (or _Insert_ > _Code cell_).

Have fun!

<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />

In [ ]:
using Pkg

In [ ]:
Pkg.add(url="https://github.com/JuliaQuantumControl/GRAPELinesearchAnalysis.jl.git")

     Cloning git-repo `https://github.com/JuliaQuantumControl/GRAPELinesearchAnalysis.jl.git`
    Updating git-repo `https://github.com/JuliaQuantumControl/GRAPELinesearchAnalysis.jl.git`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed x265_jll ───────────────────────── v3.5.0+0
   Installed Calculus ───────────────────────── v0.5.1
   Installed libfdk_aac_jll ─────────────────── v2.0.2+0
   Installed GR_jll ─────────────────────────── v0.72.4+0
   Installed Libmount_jll ───────────────────── v2.35.0+0
   Installed JpegTurbo_jll ──────────────────── v2.1.91+0
   Installed OffsetArrays ───────────────────── v1.12.9
   Installed HypergeometricFunctions ────────── v0.3.15
   Installed LERC_jll ───────────────────────── v3.0.0+1
   Installed StatsFuns ──────────────────────── v1.3.0
   Installed Opus_jll ───────────────────────── v1.3.2+0
   Installed Xorg_xkbcomp_jll ───────────────── v1.4.2+4
   Installed LoggingExtras ───────────

In [ ]:
Pkg.add("QuantumControl")

   Resolving package versions...
   Installed QuantumControl ─ v0.7.0
   Installed Krotov ───────── v0.5.3
    Updating `~/.julia/environments/v1.8/Project.toml`
  [8a270532] + QuantumControl v0.7.0
    Updating `~/.julia/environments/v1.8/Manifest.toml`
  [b05dcdc7] + Krotov v0.5.3
  [8a270532] + QuantumControl v0.7.0
Precompiling project...
  ✓ Krotov
  ✓ QuantumControl
  2 dependencies successfully precompiled in 12 seconds. 269 already precompiled.


In [ ]:
using GRAPELinesearchAnalysis

In [ ]:
using QuantumControl

In [ ]:
Pkg.add("QuantumControlBase")

   Resolving package versions...
    Updating `~/.julia/environments/v1.8/Project.toml`
  [f10a33bc] + QuantumControlBase v0.8.2
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [ ]:
using QuantumControlBase

In [ ]:
ϵ1(t) = 0.2 * QuantumControl.Shapes.flattop(t, T=5, t_rise=0.3, func=:blackman);
ϵ2(t) = 0.2 * QuantumControl.Shapes.flattop(t, T=5, t_rise=0.3, func=:blackman);

In [ ]:
"""Two-level-system Hamiltonian."""
function hamiltonian(Ω=1.0, ϵ1=ϵ1,ϵ2=ϵ2 )
    σ̂_z = ComplexF64[
        1  0
        0 -1
    ]
    σ̂_x = ComplexF64[
        0  1
        1  0
    ]
    σ̂_y = ComplexF64[
        0  0-1im
        0+1im  0
    ]
    Ĥ₀ = -0.5 * Ω * σ̂_z
    Ĥ₁ = σ̂_x
    Ĥ2 = σ̂_y
    return (Ĥ₀, (Ĥ₁, ϵ1), (Ĥ2, ϵ2))
end;

In [ ]:
H = hamiltonian();

In [ ]:
tlist = collect(range(0, 5, length=500));

In [ ]:
using LinearAlgebra

In [ ]:
function ket_(label)
    result = Dict("0" => Vector{ComplexF64}([1, 0]), "1" => Vector{ComplexF64}([0, 1]))
    return result[string(label)]
end;

In [ ]:
basis = [ket_("0"),ket_("1")];

In [ ]:
basis_tgt=Array[]
for i in 1:100
   b= transpose(RandU[i]) * basis
   push!(basis_tgt, b)

end


In [ ]:
objectives=Array[]
for i in 1:100
   obj = [Objective(initial_state = Ψ, target_state = Ψtgt, generator = H) for
    (Ψ, Ψtgt) ∈ zip(basis, basis_tgt[i])];
    push!(objectives, obj)
end




In [ ]:
problem=[]
for i in 1:100

    prob = ControlProblem(
    objectives=objectives[i],
    tlist=tlist,
    pulse_options=Dict(),
    iter_stop=500,
    J_T=QuantumControl.Functionals.J_T_sm,
    check_convergence=res -> begin
        ((res.J_T < 1e-3) && (res.converged = true) && (res.message = "J_T < 10⁻³"))
    end,
    );
    push!(problem, prob)
end

In [ ]:
Pkg.add("QuantumPropagators")
using QuantumPropagators

   Resolving package versions...
    Updating `~/.julia/environments/v1.8/Project.toml`
  [7bf12567] + QuantumPropagators v0.5.0
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [ ]:
function store_pulses(wrk, iteration, _...)
    L = length(wrk.controls)
    ϵ_opt = reshape(wrk.pulsevals, L, :)
    opt_pulses = [QuantumControl.Controls.discretize_on_midpoints(ϵ_opt[l, :], tlist) for l=1:L]
    return Tuple(opt_pulses)
end

store_pulses (generic function with 1 method)

In [ ]:
Pkg.add("GRAPE")
using GRAPE

   Resolving package versions...
    Updating `~/.julia/environments/v1.8/Project.toml`
  [6b52fcaf] + GRAPE v0.5.4
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [ ]:
wrk=[]
for i in 1:100
  wrkk=GRAPE.GrapeWrk(problem[i]; )
  push!(wrk, wrkk)
end


In [ ]:
results=[]

for i in 1:100
  opt_result_LBFGSB = optimize(

      problem[i],
      method = :grape,
      force = true,
      info_hook = QuantumControlBase.chain_infohooks(

          QuantumControl.GRAPE.print_table,
          store_pulses
      )
  );
  push!(results, opt_result_LBFGSB)
end

#opt_result_LBFGSB

 iter.        J_T     |∇J_T|       ΔJ_T   FG(F)    secs
     0   6.76e-01   1.59e-01        n/a    1(0)     9.9
     1   6.51e-01   1.61e-01  -2.54e-02    1(0)     2.0
     2   1.69e-01   1.12e-01  -4.81e-01    3(0)     0.5
     3   4.55e-02   7.59e-02  -1.24e-01    2(0)     0.3
     4   5.90e-04   9.19e-03  -4.49e-02    1(0)     0.1
 iter.        J_T     |∇J_T|       ΔJ_T   FG(F)    secs
     0   8.75e-01   1.37e-01        n/a    1(0)     0.2
     1   8.55e-01   1.46e-01  -1.94e-02    1(0)     0.1
     2   4.32e-02   6.09e-02  -8.12e-01    5(0)     0.8
     3   1.43e-02   2.73e-02  -2.89e-02    2(0)     0.4
     4   4.40e-03   2.06e-02  -9.91e-03    1(0)     0.2
     5   1.98e-03   1.63e-02  -2.42e-03    1(0)     0.2
     6   1.79e-06   5.22e-04  -1.98e-03    1(0)     0.3
 iter.        J_T     |∇J_T|       ΔJ_T   FG(F)    secs
     0   9.11e-01   8.92e-02        n/a    1(0)     0.4
     1   9.03e-01   9.31e-02  -8.12e-03    1(0)     0.2
     2   2.51e-02   5.60e-02  -8.78e-01    4(0) 

In [ ]:
N=100
J_F_avg_const_2G=0
sum=0
for i in 1:N

  sum=J_F_const_2G[i][end] + sum

end
J_F_avg_const_2G= sum/N

J_F_avg_const_2G

0.00026330660000000003

In [ ]:
t_conv_const_2G=[]

for i in 1:100
  t_sec = 0
  t_conv_i = []
   for j = 1: length(iter_sec_const_2G[i])
      t_sec = iter_sec_const_2G[i][j] + t_sec
      push!(t_conv_i, t_sec)
  end
  push!(t_conv_const_2G, t_conv_i)
end
println(t_conv_const_2G)

Any[Any[9.9, 11.9, 12.4, 12.700000000000001, 12.8], Any[0.2, 0.30000000000000004, 1.1, 1.5, 1.7, 1.9, 2.1999999999999997], Any[0.4, 0.6000000000000001, 1.3, 1.9, 2.1999999999999997], Any[0.3, 0.5, 1.2, 1.5, 1.6, 1.9000000000000001], Any[0.1, 0.2, 1.1, 1.6, 1.7000000000000002, 1.9000000000000001, 2.1], Any[0.2, 0.4, 1.1, 1.8, 1.9000000000000001, 2.0, 2.2], Any[0.1, 0.2, 0.8, 0.9, 1.0, 1.1, 1.2000000000000002], Any[0.1, 0.2, 0.5, 0.6, 0.7, 0.7999999999999999, 0.8999999999999999], Any[0.1, 0.2, 0.6000000000000001, 0.7000000000000001, 0.8, 0.9, 1.0, 1.1, 1.2000000000000002], Any[0.1, 0.2, 0.5, 0.7], Any[0.1, 0.2, 0.4, 0.6000000000000001, 0.7000000000000001], Any[0.2, 0.30000000000000004, 0.8, 1.0, 1.1, 1.3], Any[0.1, 0.2, 0.5, 0.7, 0.8999999999999999, 0.9999999999999999, 1.0999999999999999, 1.2], Any[0.1, 0.2, 0.4, 0.6000000000000001, 0.7000000000000001, 0.8], Any[0.1, 0.2, 0.6000000000000001, 0.8, 0.9, 1.0], Any[0.1, 0.2, 0.6000000000000001, 0.8, 0.9, 1.0, 1.1], Any[0.1, 0.2, 0.4, 0.60000

In [ ]:
N=100
t_conv_avg_const_2G=0
sum=0
for i in 1:N

  sum=t_conv_const_2G[i][end] + sum

end

t_conv_avg_const_2G= sum/N
println(t_conv_avg_const_2G)

1.048
